### Test infomap on the generated LFR benchmark graphs


Import necessary packages and set up plot formatting.

In [1]:
from collections import namedtuple

import matplotlib.pyplot as plt

from src.lfr.benchmark_results import *
%pylab

plt.rcParams.update({'font.size': 15})
plt.rcParams.update({'text.usetex': True})
plt.rcParams.update({'font.family': 'sans-serif'})
plt.rcParams.update({'lines.linewidth': 2})
plt.rcParams.update({'lines.markersize': 8})
plt.rcParams.update({'lines.markeredgewidth': 2})
plt.rcParams.update({'axes.labelpad': 15})
plt.rcParams['text.latex.preamble'] = [
    r'\usepackage{amsmath,amssymb,amsfonts,amsthm}',
    r'\usepackage{siunitx}',  # i need upright \micro symbols, but you need...
    r'\sisetup{detect-all}',  # ...this to force siunitx to actually use your fonts
    r'\usepackage{helvet}',  # set the normal font here
    r'\usepackage{sansmath}',  # load up the sansmath so that math -> helvet
    r'\sansmath'  # <- tricky! -- gotta actually tell tex to use!
]

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


Plot benchmark results for a fixed network size $n$ as a function of the mixing parameter $\mu$.



In [10]:
n = 1200 # fixed network size of the benchmark series
plot_uncertainty = True # if True plot uncertainty as a margin of one std deviation
# assemble a list of results to plot (path + plot formatting)
ResultInfo = namedtuple('ResultInfo', ['path', 'label', 'linestyle'])
info = []
info.append(ResultInfo(f'../results/lfr/infomap/{n}n.pkl', 'Infomap', 'x:'))
info.append(ResultInfo(f'../results/lfr/synwalk/{n}n.pkl', 'SynWalk', '^:'))
info.append(ResultInfo(f'../results/lfr/walktrap/{n}n.pkl', 'Walktrap', 's:'))
info.append(ResultInfo(f'../results/lfr/label_propagation/{n}n.pkl', 'Label Propagation', 'o:'))

# plot the results
plt.close('all')
plt.figure(figsize=(8,6))

for entry in info:
    results = BenchmarkResults.load(entry.path)
    xdata = results.get_var_list()
    ydata = results.get_mean_scores()
    plt.plot(xdata, ydata, entry.linestyle, label=entry.label)

    if plot_uncertainty:
        upper = np.clip(ydata + results.get_score_std(), 0.0, 1.0)
        lower = np.clip(ydata - results.get_score_std(), 0.0, 1.0)
        plt.fill_between(xdata, upper, lower, alpha=0.25)

plt.title(f'LFR benchmark - {n} nodes')
plt.xlabel('Mixing parameter, $\mu$')
plt.ylabel(r'Adjusted Mutual Information, $\mathcal{I}^{adj}(\mathcal{Y}, \mathcal{Y}_{true})$')
plt.legend(loc=1)